First we load our mesh. We will be using kaolin and our bunny model.

In [1]:
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

mesh = kal.rep.TriangleMesh.from_obj("bunny.obj")

Now, we can compute the normals at each vertex.

In [2]:
import normals_lib
normals = normals_lib.compute_simple_normals(mesh)
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=normals.numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Next, we calculate weights, called pointareas and cornerareas, which we will later
use to calculate the curvature of the mesh.

In [3]:
import pointareas as pareas
pointareas, cornerareas = pareas.compute_pointareas(mesh)

Now we can calculate the principal curvatures and directions. If the normals, pointareas and cornerareas are
not passed as parameters, _compute_curvatures_ will calculate them.

In [4]:
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

# You can also use torch.lstsq to compute curvatures, but cholesky is the recommended method.
# k1, k2, e1, e2 = curvature.compute_curvatures(mesh, method='lstsq', normals=normals, pointareas=pointareas,
#                                               cornerareas=cornerareas)

Next, we can calculate the derivative of curvature tensor (__C__) at each vertex of the mesh.

In [5]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

We can also calculate the radial curvature and directional derivatives of the radial curvature. There is a default camera position, but one can also be passed as a parameter. It must be a torch tensor of size 3.

In [6]:
DwKr, kr = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc)

We can also calculate the sctest value that is used in _rtsc_ for drawing Suggestive Contours.
As with _compute_DwKr_, a camera position can be defined.

In [7]:
import perview

ndotv, krf, _, sctest = perview.compute_perview(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc)

We threshold the values of the results in order to allow for easier visualization.

In [8]:
kupper = int(k1.shape[0] * 0.95)
klower = int(k1.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(k1, kupper)
dwlower, _ = torch.kthvalue(k1, klower)
k1[k1 > dwupper] = dwupper
k1[k1 < dwlower] = dwlower
kupper = int(k2.shape[0] * 0.95)
klower = int(k2.shape[0] * 0.05)
krupper, _ = torch.kthvalue(k2, kupper)
krlower, _ = torch.kthvalue(k2, klower)
k2[k2 > krupper] = krupper
k2[k2 < krlower] = krlower

In [9]:
kupper = int(DwKr.shape[0] * 0.95)
klower = int(DwKr.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(DwKr, kupper)
dwlower, _ = torch.kthvalue(DwKr, klower)
DwKr[DwKr > dwupper] = dwupper
DwKr[DwKr < dwlower] = dwlower
kupper = int(kr.shape[0] * 0.95)
klower = int(kr.shape[0] * 0.05)
krupper, _ = torch.kthvalue(kr, kupper)
krlower, _ = torch.kthvalue(kr, klower)
kr[kr > krupper] = krupper
kr[kr < krlower] = krlower

In [10]:
kupper = int(sctest.shape[0] * 0.95)
klower = int(sctest.shape[0] * 0.05)
scupper, _ = torch.kthvalue(sctest, kupper)
sclower, _ = torch.kthvalue(sctest, klower)
sctest[sctest > scupper] = scupper
sctest[sctest < sclower] = sclower

kupper = int(krf.shape[0] * 0.95)
klower = int(krf.shape[0] * 0.05)
scupper, _ = torch.kthvalue(krf, kupper)
sclower, _ = torch.kthvalue(krf, klower)
krf[krf > scupper] = scupper
krf[krf < sclower] = sclower

Now we can visualize our results on the mesh using meshplot.

In [11]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [12]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [13]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [14]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…